<a href="https://colab.research.google.com/github/enginearn/llm_book_for_intro/blob/main/llm_book_for_intro_ch_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 1

In [2]:
!pip install transformers[ja,sentencepiece,torch]

In [3]:
from transformers import pipeline

## Text classification

In [4]:
from transformers.tools import text_classification
text_classification_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-marc_ja"
)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
positive_text = "明日は明日の風が吹く"
print(text_classification_pipeline(positive_text)[0])

{'label': 'positive', 'score': 0.998440682888031}


In [6]:
negative_text = "お金がなさ過ぎてつらい..."
print(text_classification_pipeline(negative_text)[0])

{'label': 'negative', 'score': 0.9948321580886841}


## Natural Language Inference: 自然言語推論

In [7]:
nli_pipline = pipeline(model="llm-book/bert-base-japanese-v3-jnli")

`entailment`は`含意`であり、"二人の女性が山を眺めています"が成立するならば、"山を眺めている人が2人います"も成立するという関係を表す。

In [8]:
text = "二人の女性が山を眺めています"
entailment_text = "山を眺めている人が2人います"

print(nli_pipline({"text": text, "text_pair": entailment_text}))

{'label': 'entailment', 'score': 0.9933545589447021}


`contradiction`は、`矛盾`

In [9]:
contradiction_text = "女性２人が山を破壊しています"

print(nli_pipline({"text": text, "text_pair": contradiction_text}))

{'label': 'contradiction', 'score': 0.9771778583526611}


In [10]:
neutral_text = "女性２人が山で料理をしています"

print(nli_pipline({"text": text, "text_pair": neutral_text}))

{'label': 'neutral', 'score': 0.9970123767852783}


## Semantic textual similarity: STS

二つのテキストが似ている度合いをスコアとして予測するタスク

In [11]:
text_sts_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-jsts",
    function_to_apply="none"
)

In [12]:
text = "マチュピチュにはマヤ文明がありました"
sim_text = "ストーンヘンジにはドルイド文明が存在していました。"

result = text_sts_pipeline({"text": text, "text_pair": sim_text})
print(result["score"])

1.0985182523727417


In [13]:
sim_text = "マチュピチュはマヤ文明の一部でした。"

result = text_sts_pipeline({"text": text, "text_pair": sim_text})
print(result["score"])

3.884174346923828


In [14]:
dissim_text = "トイレの壁に黒いタオルがかけられています"

result = text_sts_pipeline({"text": text, "text_pair": dissim_text})
print(result["score"])

-0.05538347363471985


In [15]:
from torch.nn.functional import cosine_similarity

sim_enc_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-unsup-simcse-jawiki",
    task="feature-extraction",
)

In [16]:
# textとsim_textのベクトルを獲得
text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]
# textとsim_textの類似度を計算
sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

0.9352689385414124


In [17]:
# dissim_textのベクトルを獲得
dissim_emb = sim_enc_pipeline(dissim_text, return_tensors=True)[0][0]
# textとdissim_textの類似度を計算
dissim_pair_score = cosine_similarity(text_emb, dissim_emb, dim=0)
print(dissim_pair_score.item())

0.33561891317367554


## Named entity recognition: NER

テキストに含まれる固有表現を抽出するタスク

In [18]:
from pprint import pprint

ner_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-ner-wikipedia-dataset",
    aggregation_strategy="simple",
)

In [19]:
text = "大谷翔平は岩手県水沢市出身のプロ野球選手です"
pprint(ner_pipeline(text))

[{'end': None,
  'entity_group': '人名',
  'score': 0.99822515,
  'start': None,
  'word': '大谷 翔平'},
 {'end': None,
  'entity_group': '地名',
  'score': 0.9987048,
  'start': None,
  'word': '岩手 県 水沢 市'}]


## Summarization generation

In [20]:
text2text_pipeline = pipeline(
    model="llm-book/t5-base-long-livedoor-news-corpus"
)

In [21]:
article = """
生成型AIは、テキスト、画像、音楽、さらには映画など、広範なデータ種類に対して複雑なパターンを学習し、新たなコンテンツを創出することが可能です。GPT-4のような最新のモデルは、既に驚くべき精度と多様性を持っていますが、さらなる進化が期待されます。
まず、生成型AIのモデルはより洗練され、特定のタスクや要求に対する独自の「スタイル」を持つことが可能になるでしょう。これは、ユーザーがAIに「詩人のように書く」や「特定のアーティストの風格で絵を描く」など、より具体的な要求をすることを可能にします。
次に、モデルは予測の質と適用範囲をさらに向上させることが期待されます。例えば、個々の言葉やフレーズだけでなく、全体的な会話や物語の文脈を理解し、それに対応した生成を行うことが可能になるでしょう。
さらに、生成型AIは個々のユーザーとの対話から学習し、ユーザーの嗜好や要求により適応する能力を強化する可能性があります。これにより、個々のユーザーにとってより有用でパーソナライズされた体験が提供されます。
最後に、AIの進化は倫理的な問題も提起します。AIが創造するコンテンツの著作権やプライバシー保護、偏見の排除など、新たな課題に立ち向かうためのガイドラインや規制の必要性が強まってきています。これらの課題を解決することで、生成型AIの今後はさらに明るいものとなるでしょう。
"""

print(text2text_pipeline(article)[0]["generated_text"])

進化するAI GPT-4 AIの未来【デジ通】


- machine translation
- dialogue system
- morphological analysis
- parsing
- coreference resolution

## Transformers

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("abeja/gpt2-large-japanese")

tokenizer.tokenize("今日は天気が良いので")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


['▁', '今日', 'は', '天気', 'が良い', 'の', 'で']

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "abeja/gpt2-large-japanese"
)

inputs = tokenizer("今日は天気が良いので", return_tensors="pt")
outputs = model.generate(
    **inputs,
    max_length=30,
    pad_token_id=tokenizer.pad_token_id
)

generated_text = tokenizer.decode(
    outputs[0],
    skip_special_tokens=True
)

print(generated_text)